<a href="https://colab.research.google.com/github/Haseon/language_detection_test/blob/master/language_detect_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 통계적기계학습 기말 프로젝트 보고서 : 201401381 박하선
#LSTM 언어 모형을 이용한 자동 언어 인식



##	1. 동기 및 이론적 배경

구글번역기가 가진 한가지 흥미로운 기능은 ‘언어 식별’이다. 무슨 언어에서 번역하는지 지정하지 않아도, ‘자동 언어 식별’을 선택하면 알아서 언어를 식별해서 원하는 언어로 번역해준다. 이런 기능은 어떻게 만든 것일까? 구글 번역기는 100개가 넘는 언어의 번역을 지원하기 때문에 일일이 수작업으로 프로그래밍한 것은 아닐 것이며, 이 기능은 기계학습을 이용해 만들어졌을 것이다. 한편 번역기의 원리를 생각해보면 이 기능은 따로 애써서 만든 것이 아니라 번역기 구축 과정에서 만들어지는 부산물을 자연스럽게 활용한 것에 불과하다고 추측했다. 기계번역과 음성인식, 문서 요약을 비롯한 많은 자연어처리 기술이 베이즈 정리를 응용한 다음과 같은 형태의 수식에 기반하고 있기 때문이다. 

![대체텍스트](https://wikimedia.org/api/rest_v1/media/math/render/svg/82c067b990481e9bab9a8ae4cee80f1dc140779c)
 
기계번역에서는 f가 출발어 표현, e가 도착어 표현이다. p(e)는 도착어에서 표현 e가 등장할 확률이며, 이것의 추정치를 제공하는 모형을 언어 모델이라고 한다. 언어 모델은 자연어처리의 기반 기술이며, 별도의 레이블이 필요 없이도 학습할 수 있는 데다가, 잘 학습시켜놓으면 수많은 과제의 성능을 끌어올려준다. 그런데 언어의 종류가 l이라고 할 때 다음과 같은 점을 생각하면 언어 모델을 자동 언어 인식에도 적용할 수 있다는 것을 알게 된다. 
 
 ![대체텍스트](https://raw.githubusercontent.com/Haseon/language_detection_test/master/image/CodeCogsEqn.gif)
 
여기서 p(l|e)는 해당 표현 e가 언어 l일 확률이다. 이는 베이즈 정리에 의해 p(e|l)p(l)/p(e)와 같은데 p(e)는 직접적으로 추정하기가 어렵고, l과 무관하게 고정되어 있기 때문에 p(l|e)가 최대가 되는 l을 찾는 문제에서는 무시할 수 있다. 또한 p(l)도 균일분포라고 가정해 무시할 수 있다. (적어도 실험에서는 그렇게 만들 수 있다.) 따라서 p(l|e)은 언어에서 해당 표현이 등장할 확률 p(e|l)에 비례하며, 해당 표현에 가장 높은 확률을 부여하는 언어 모델의 언어로 분류할 수 있다. 

언어 모델을 구축하는 방법은 여러가지가 있지만 현재 가장 높은 성능을 보여주는 방법론은 신경망을 이용한 방법이며 이 글에서도 자연어 텍스트와 같은 시계열적 데이터를 처리하는 데 많이 이용되는 Long-Term Short Memory를 이용하여 언어 모델을 구축할 것이다. 이후 언어 모델을 활용해서 분류 작업을 수행할 수 있는지, 그 성능이 얼마나 높은지 살펴볼 것이다. 


## 2. 구현과 데이터, 실험

Keras를 사용해 구현했다. 분류할 언어로는 영어, 네덜란드어, 라틴어, 우즈벡어, 인도네시아어, 타갈로그어, 스와힐리어를 선정했다. 이 언어들은 모두 영어에서 사용하는 문자만을 주로 사용하는 언어들이다. 거의영어에서 사용하지 않는 문자를 사용하는 언어(한국어, 프랑스어, 스페인어, 아랍어 등)는 특정 문자의 존재를 이용해서 너무 간단히 언어를 분류할 수 있게 되기 때문에 LSTM 언어 모델을 쓰는 의미가 퇴색된다고 판단했기 때문이다. GB 단위의 대량의 코퍼스를 사용할 수도 있지만 학습 시간이 너무 길기 때문에, 위키백과의 문서를 최대한 다양하고 균형잡힌 주제로 선택하여 사용했다. 훈련 데이터로는 [철학, 과학, 기술, 정치, 경제, 예술, 인권, 문화, 역사, 언어, 문학, 종교, 지리, 수학, 스포츠]를 표제어로 하는 문서를 언어별로 추출했고, 시험 데이터로는 [한국, 대학, 통계, 화폐]를 표제어로 하는 문서를 언어별로 추출했다. 자료는 [이곳](https://github.com/Haseon/language_detection_test/tree/master/corpus)에서 열람할 수 있다.

먼저 필요한 라이브러리들을 불러온다. 그리고 언어 목록 및 인공신경망의 one-hot 벡터로 표상할 수 있는 문자 집합을 charlist로 정의하고, 그에 해당하지 않는 문자를 검색하는 정규표현식을 regex로 정의한다. 

In [0]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
from os.path import isfile
import random
import sys
import codecs
import re
import pickle
from math import log, exp

language_list = ["english", "dutch", "latin", "uzbek", "indonesian", "tagalog", "swahili"]
regex = "[^ \n\t\"\'\(\),\-\_!\?\/\.0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ\[\]abcdefghijklmnopqrstuvwxyz\ufeff]"
charlist = " \n\t\"\'(),-_!?/.0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz\ufeff"

그리고 위키백과에서 추출한 데이터를 언어별로 불러오고 전처리한다. 

In [0]:
train_corpora = {}
test_corpora = {}
model_dic = {}

def prepare_corpora():
  for l in language_list:
    for mode in ['corpus', 'test']:
      path = get_file(
          f'{l}_{mode}.txt',
          origin=f'https://raw.githubusercontent.com/Haseon/language_detection_test/master/corpus/{l}_{mode}.txt')
      with codecs.open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        #charlist에 없는 문자는 언어 별 자료의 양질에 따른 차이이므로(외국어 문자, 특수 문자 등의 차이) 제거합니다.
        text=re.sub(regex, "_", text)
        (train_corpora, test_corpora)[mode=='test'].update({l: text})
      print("corpus of:", l, mode)
      print('코퍼스 길이:', len(text)) 
      print(len(set(text)), len(charlist))

def save_prepro_corpora():
  with open("prepro_corpora.p", "wb") as f:
      pickle.dump((train_corpora, test_corpora), f)

def load_prepro_corpora():
    with open("prepro_corpora.p", "rb") as f:
        train_corpora, test_corpora = pickle.load(f)
    return train_corpora, test_corpora


char_indices = dict((c, i) for i, c in enumerate(charlist))
indices_char = dict((i, c) for i, c in enumerate(charlist))

if isfile("prepro_corpora.p"):
  train_corpora, test_corpora = load_prepro_corpora()
else:
  prepare_corpora()
  save_prepro_corpora()

그리고 해당 데이터에 대하여 LSTM 신경망을 이용해 훈련시킨다. 텍스트는 one-hot 벡터로 변환하여 처리하며, 신경망 최적화 방법으로는 학습률 0.01로 시작하는 RMSprop를, 비용 함수로는 교차 엔트로피(CE)를 사용한다. 교차 엔트로피를 사용하는 것은 이것이 이전의 텍스트를 이용해서 다음 문자를 예측하는, 분류 문제이기 때문이다. 

---



In [5]:
def train(l):
  print("train:", l)
  maxlen = 40
  step = 3
  sentences = []
  next_chars = []
  text = train_corpora[l]
  for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
  print('nb sequences:', len(sentences))

  print('Vectorization...')
  x = np.zeros((len(sentences), maxlen, len(charlist)), dtype=np.bool)
  y = np.zeros((len(sentences), len(charlist)), dtype=np.bool)
  for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
      x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


  # build the model: a single LSTM
  print('Build model...')
  model = Sequential()
  model.add(LSTM(128, input_shape=(maxlen, len(charlist))))
  model.add(Dense(len(charlist), activation='softmax'))
  
  optimizer = RMSprop(lr=0.01)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer)
  
  model.fit(x, y,
            batch_size=128,
            epochs=1)
  return model
  
def make_model():
  for l in language_list:
    model = train(l)
    model_dic.update({l: model})

def save_model():
  with open("model_dic.p", "wb") as f:
    pickle.dump(model_dic, f)

def load_model():
  with open("model_dic.p", "rb") as f:
    return pickle.load(f)

def posttrain():
  pass

if model_dic == {}:
  make_model()
  save_model()
else:
  load_model()

train: english
nb sequences: 138356
Vectorization...
Build model...
Epoch 1/1
138356/138356 [==============================] - 73s 525us/step - loss: 2.1156
train: dutch
nb sequences: 101606
Vectorization...
Build model...
Epoch 1/1
101606/101606 [==============================] - 54s 536us/step - loss: 2.1786
train: latin
nb sequences: 52821
Vectorization...
Build model...
Epoch 1/1
52821/52821 [==============================] - 30s 560us/step - loss: 2.4329
train: uzbek
nb sequences: 70439
Vectorization...
Build model...
Epoch 1/1
70439/70439 [==============================] - 39s 552us/step - loss: 2.4171
train: indonesian
nb sequences: 159394
Vectorization...
Build model...
Epoch 1/1
159394/159394 [==============================] - 85s 536us/step - loss: 1.9760
train: tagalog
nb sequences: 74622
Vectorization...
Build model...
Epoch 1/1
74622/74622 [==============================] - 41s 556us/step - loss: 2.0055
train: swahili
nb sequences: 32977
Vectorization...
Build model...
Epo

시험 삼아 다음과 같은 코드로 각 언어의 모델에 따라 텍스트를 생성해보면 다음과 같다. 온도(temperature)가  낮을수록 확률이 높은 선택지만을 선택하고, 온도가 높을수록 확률이 낮은 선택지도 골고루 선택한다. 생성 결과 의미는 통하지 않지만 그 언어스러운 텍스트를 생성하여 어느 정도 언어의 분포를 잘 파악하는 것으로 보인다. 말이 전혀 되지 않는 문장인 것은 Epoch이 1번뿐인 데에서 기인하는 한계로, 더 많은 훈련을 통해 개선할 수 있을 것으로 보인다. 

In [7]:
def sample(preds, temperature=1.0):
    # 확률 배열로부터 문자의 주소를 확률에 따라 추출하는 함수
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


  
def generate_text(l):
    print("Language:", l)
    text = test_corpora[l]
    maxlen = 40
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(charlist)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model_dic[l].predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

for l in language_list:
    generate_text(l)

Language: english
----- diversity: 0.2
----- Generating with seed: "n. Other large groups of Korean speakers"
n. Other large groups of Korean speakers are the resears and the sunciplition of the some the political artical of the socies of the souction and the supporal to the souction of the soude and the some to the poster and the some articled to the most to the resears of the political scientistical artical artical artical of the sourtions and the souded to the political science of the relations are such as the poster and the political artica
----- diversity: 0.5
----- Generating with seed: "n. Other large groups of Korean speakers"
n. Other large groups of Korean speakers such as the soude of the socces to than incoures to the poind the reconery as the somen bes the sumplical producents to mathomatical producents and more of the more to procession of the natural beconomic bast and allogy of the reconce to the les of son a compurition from the mother that to the contrints of the rese

이제 랜덤으로 ```totaln```개의 표현을 시험데이터에서 추출해 분류한다.

In [6]:
def test_detection():
    totaln = 100
    accurate = 0 
    ll = len(language_list)
    ld = dict({l:i for i,l in enumerate(language_list)})
    conf_mat = np.zeros((ll, ll), dtype=int)
    for _ in range(totaln):
        true_l = random.choice(language_list)
        text = test_corpora[true_l]
        testlen = 100
        maxlen = 40
        start_index = random.randint(0, len(text) - testlen - 1)
        sentence = text[start_index: start_index + testlen]
        x = np.zeros((1, maxlen+testlen, len(charlist)))
        y = np.zeros((testlen, len(charlist)))
        logprobs = {}
        for cand_l in language_list:
            logprobs.update({cand_l:1})
            for i in range(testlen):
                x[0, maxlen+i, char_indices[sentence[i]]] = 1.
            for i in range(testlen):
                charix = char_indices[sentence[i]]
                y[i, charix] = 1.

                x_pred = x[:, i:i+maxlen]
                y_pred = y[i]


                preds = model_dic[cand_l].predict(x_pred, verbose=0)[0]
                preds = preds[charix]
                cond_prob = preds
                logprobs[cand_l] += log(cond_prob)
                n = (i*20)//(testlen-1)
                done = '='*n
                todo = '.'*(20-n)
                spaces = ' '*10
                print(f"{_} true_l:{true_l} cand_l:{cand_l} [{done}{todo}]{spaces}", end='\r')
        print()
        vsum=sum(exp(v) for v in logprobs.values())
        print(' '.join(f"{l}: {exp(logprobs[l])/vsum}" for l in language_list))
        pred_l = max(logprobs, key=logprobs.get)
        conf_mat[ld[true_l], ld[pred_l]] += 1
        right = true_l == pred_l
        if right:
            accurate += 1
        print("predicted:", pred_l, ['***', ''][right])
        print("accuracy so far:", accurate/(_+1))
        if _ % 10 == 0:
            print(conf_mat)
    print("final accuracy:", accurate/totaln)

test_detection()


0 true_l:dutch cand_l:swahili [====================]          
english: 4.11581102287201e-23 dutch: 1.0 latin: 1.6264069127075744e-30 uzbek: 1.7132725202805163e-47 indonesian: 3.782704794583116e-49 tagalog: 4.399752062860427e-33 swahili: 8.59798465099219e-55
predicted: dutch 
accuracy so far: 1.0
[[0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
1 true_l:dutch cand_l:swahili [====================]          
english: 1.0688510235157355e-82 dutch: 1.0 latin: 7.460666411230727e-87 uzbek: 4.804897854961664e-79 indonesian: 9.091014978108364e-83 tagalog: 6.029116194354087e-84 swahili: 1.0635922663846357e-78
predicted: dutch 
accuracy so far: 1.0
2 true_l:dutch cand_l:swahili [====================]          
english: 4.147670546530908e-33 dutch: 1.0 latin: 5.958561204368052e-31 uzbek: 2.5622250884254567e-58 indonesian: 2.9070941392911266e-40 tagalog: 5.073366147668347e-37 swahili: 1.9134105739905923e-49
predicted: dutch 
ac

스와힐리어가 인도네시아어로 오분류된 단 한 개의 경우를 제외하면 모두 성공적으로 분류하여, 99%의 높은 정확도로 분류함을 확인할 수 있다. 